In [ ]:
import cv2
import os
from google.colab import drive
from google.colab.patches import cv2_imshow

def extract_frames(video_path, output_folder, max_frames=5):
    # Google Driveをマウント
    drive.mount('/content/drive')

    # ビデオファイルを開く
    video = cv2.VideoCapture(video_path)

    # 出力フォルダが存在しない場合は作成
    full_output_path = f"/content/drive/MyDrive/{output_folder}"
    if not os.path.exists(full_output_path):
        os.makedirs(full_output_path)

    # フレームカウンタ
    frame_count = 0

    frames = []

    while frame_count < max_frames:
        # フレームを読み込む
        success, frame = video.read()

        if not success:
            break

        # フレームを保存
        output_path = os.path.join(full_output_path, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(output_path, frame)

        # フレームを表示（Colab用）
        cv2_imshow(frame)

        frame_count += 1

        frames.append(frame)

    # ビデオファイルを閉じる
    video.release()

    print(f"合計 {frame_count} フレームを抽出しました。")
    print(f"フレームは {full_output_path} に保存されました。")

    return frames




In [ ]:
import cv2
import numpy as np
import base64
from PIL import Image
import io
from google.colab.patches import cv2_imshow

def detect_and_inpaint_faces(frame):
    """
    フレームから顔を検出し、inpaintingを行う関数
    Args:
        frame: cv2で読み込んだ画像（numpy.ndarray）
    Returns:
        processed_image: 処理後の画像（numpy.ndarray）
        face_mask: 顔領域のマスク（numpy.ndarray）
    """
    # 顔検出器の読み込み
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # グレースケールに変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔検出
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # マスクの作成（顔領域を白、それ以外を黒に）
    face_mask = np.zeros(frame.shape[:2], dtype=np.uint8)

    # 検出された顔領域を少し大きめに取る
    for (x, y, w, h) in faces:
        # マスクを少し大きめに
        padding = int(min(w, h) * 0.1)  # パディングを10%に設定
        x1 = max(x - padding, 0)
        y1 = max(y - padding, 0)
        x2 = min(x + w + padding, frame.shape[1])
        y2 = min(y + h + padding, frame.shape[0])

        # マスクに顔領域を追加
        cv2.rectangle(face_mask, (int(x1), int(y1)), (int(x2), int(y2)), 255, -1)

    # インペインティングの実行
    processed_image = cv2.inpaint(frame, face_mask, 3, cv2.INPAINT_TELEA)

    return processed_image, face_mask

def prepare_for_imagen(image):
    """
    画像をImagen APIで使用できる形式に変換する関数
    Args:
        image: numpy.ndarray形式の画像
    Returns:
        base64_image: Base64エンコードされた画像文字列
    """
    # OpenCV画像をPIL形式に変換
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)

    # PILイメージをバイトストリームに変換
    byte_stream = io.BytesIO()
    pil_image.save(byte_stream, format='PNG')
    byte_stream.seek(0)

    # Base64エンコード
    base64_image = base64.b64encode(byte_stream.getvalue()).decode('utf-8')

    return base64_image

def base64_to_cv2(base64_string):
    """
    Base64文字列をOpenCV画像形式に変換する関数
    """
    # Base64をデコード
    img_data = base64.b64decode(base64_string)

    # バイトデータをnumpy配列に変換
    nparr = np.frombuffer(img_data, np.uint8)

    # numpy配列をOpenCV画像に変換
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    return img

def process_frame_for_imagen(frame):
    """
    フレームを処理してImagen API用に準備する関数
    Args:
        frame: cv2で読み込んだ画像
    Returns:
        base64_processed: 処理済み画像のBase64文字列
        base64_mask: マスクのBase64文字列
    """
    # 顔検出とinpainting
    processed_image, face_mask = detect_and_inpaint_faces(frame)

    # Imagen API用にBase64エンコード
    base64_processed = prepare_for_imagen(processed_image)
    base64_mask = prepare_for_imagen(cv2.cvtColor(face_mask, cv2.COLOR_GRAY2BGR))

    return base64_processed, base64_mask

# 使用例
def main():
    # extract_framesで取得したフレームに対して処理を実行
    frame = cv2.imread("/content/drive/MyDrive/output_frames/frame_0000.jpg")

    if frame is None:
        print("画像の読み込みに失敗しました。パスを確認してください。")
        return

    # 処理の実行
    base64_processed, base64_mask = process_frame_for_imagen(frame)

    # 処理結果の確認
    processed_img = base64_to_cv2(base64_processed)
    mask_img = base64_to_cv2(base64_mask)

    # 結果の表示
    print("処理済み画像:")
    cv2_imshow(processed_img)
    print("\nマスク画像:")
    cv2_imshow(mask_img)

    return base64_processed, base64_mask

# extract_frames関数で使用する場合の例
def process_video_frame(frame):
    """
    ビデオフレームを処理する関数
    """
    base64_processed, base64_mask = process_frame_for_imagen(frame)

    # 結果の表示（必要に応じて）
    processed_img = base64_to_cv2(base64_processed)
    mask_img = base64_to_cv2(base64_mask)

    print("処理済み画像:")
    cv2_imshow(processed_img)
    print("\nマスク画像:")
    cv2_imshow(mask_img)

    return base64_processed, base64_mask

In [1]:
!pip install onnxruntime-gpu
!pip install mediapipe opencv-python-headless pillow numpy insightface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 24.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 99.5 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064899 sha256=3be7342af9bc54df94379d06db07cc4616955f757ca0c74eda766d0357813c21
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [24]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
import insightface
import matplotlib.pyplot as plt
from tqdm import tqdm

def extract_frames(video_path, num_frames=10):
    frames = []
    cap = cv2.VideoCapture(video_path)
    for i in range(num_frames):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

def display_frames(frames):
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.ravel()

    for idx, frame in enumerate(frames):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        axes[idx].imshow(frame_rgb)
        axes[idx].axis('off')
        axes[idx].set_title(f'Frame {idx+1}')

    plt.tight_layout()
    plt.show()

def process_video(source_path, video_path, output_path):
    app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    app.prepare(ctx_id=0, det_size=(640, 640))

    model = insightface.model_zoo.get_model('/content/drive/MyDrive/videoconvert/inswapper_128.onnx')

    source_img = cv2.imread(source_path)
    source_faces = app.get(source_img)
    if not source_faces:
        raise ValueError("ソース画像から顔を検出できませんでした")
    source_face = source_faces[0]

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    with tqdm(total=total_frames, desc="処理中") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            target_faces = app.get(frame)
            if target_faces:
                result = frame.copy()
                for target_face in target_faces:
                    result = model.get(result, target_face, source_face, paste_back=True)
                out.write(result)
            else:
                out.write(frame)

            pbar.update(1)

    cap.release()
    out.release()
    print(f"処理が完了しました: {output_path}")

# メイン処理
source_path = '/content/drive/MyDrive/videoconvert/hashikan2.jpeg'
video_path = '/content/drive/MyDrive/videoconvert/raw.mov'
output_path = '/content/drive/MyDrive/videoconvert/output.mp4'

process_video(source_path, video_path, output_path)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
Applied prov

処理中: 100%|██████████| 723/723 [15:57<00:00,  1.32s/it]

処理が完了しました: /content/drive/MyDrive/videoconvert/output.mp4
